# Probability integral transform (PIT) Histogram

Get observation & forecast data

In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data
from HelpFunctions.date_and_time import most_recent_thursday, split_time
from Energy.Models.baseline import baseline
from HelpFunctions.calc_score import evaluate_horizon
from HelpFunctions.mix_models import mix_models
from Energy.Models.Model1 import model1
from Energy.Models.Model2 import model2
from Energy.Models.Model4_population import model4_population
from Energy.Models.Model4 import model4
from Energy.Models.Model3 import model3
from Energy.Models.Model5 import model5
from Energy.Models.Model4_holidays_2 import model4_holidays_2
from Energy.Models.Model4_sunhours import model4_sunhours
# import importlib
# importlib.reload(Energy.Models)

In [ ]:
# from Energy.HelpFunctions.get_energy_data import fetch_energy_data
# fetch_energy_data()

In [ ]:
df = get_energy_data()
df = prepare_data(df)

In [ ]:
start_date_excl = most_recent_thursday(df)
df_cval = df.loc[df.index < start_date_excl]

Use cross validate functions to get the data. Cannot import fun

In [ ]:
from HelpFunctions.mix_models import mix_models_per_horizon


def evaluate_models(models, df, last_x, years =False, months=False, weeks=False):
    # Check that exactly one of the boolean parameters is True
    if sum([years, months, weeks]) != 1:
        raise ValueError("Exactly one of the boolean parameters (years, months, weeks) must be True.")
    
    years = int(years)
    months = int(months)
    weeks = int(weeks)
        
    for m in models:
        print(f'*********** Start the evaluation of Model {m["name"]} ***********')
        m['evaluation'] = evaluate_model(m, df, last_x, years, months, weeks)
        
def evaluate_model(model, df, last_x, years, months, weeks):
    df_before = df
    evaluation = pd.DataFrame()
    
    for w in range(last_x):
        print(f'Iteration {w} of {last_x}')
        df_before, df_after = split_time(df_before, num_years=years, num_months=months, num_weeks=weeks)        
        
        pred = None     
        # Is mixed model?
        if callable(model['function']):
            pred = model['function'](df_before)
        else:
            pred = mix_models_per_horizon(model['function'][0], model['function'][1], df_before)
               
        
        obs = pd.DataFrame({'gesamt': df.loc[pred['forecast_date']]["gesamt"]})
        pred = pred.set_index('forecast_date')
        merged_df = pd.merge(pred, obs, left_index=True, right_index=True) 
    
    
         # Add scores to the merged_df
        for index, row in merged_df.iterrows():
            quantile_preds = row[['q0.025','q0.25','q0.5','q0.75','q0.975']]
            observation = row['gesamt']
            score = evaluate_horizon(quantile_preds, observation)
            merged_df.at[index, 'score'] = score
        # print(merged_df[['q0.025','q0.25','q0.5','q0.75','q0.975']])
        evaluation = pd.concat([evaluation, merged_df])
    return evaluation

In [ ]:
from Energy.Models import mstl
models = [
    # {
    #     'name': 'baseline',
    #     'function': baseline
    # },
    # {
    #     'name': 'model5',
    #     'function': model5
    #  }
    {
        'name': 'mstl',
        'function': mstl.mstl
     },
]
evaluate_models(models, df_cval, last_x=10, weeks=True)

In [ ]:
# with open('./Model evaluations/big_evaluation.pkl', 'rb') as f:
#     models = pickle.load(f)

In [ ]:
import numpy as np


model_index = 0
results = []

for h in ['36 hour', '40 hour', '44 hour', '60 hour', '64 hour', '68 hour']:
    results.append(models[model_index]['evaluation'][models[model_index]['evaluation']['horizon'] == h])

# results36h = models[0]['evaluation'][models[0]['evaluation']['horizon'] == '36 hour']



# results = models[0]['evaluation']



## Create PIT Histogram

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import string
import random
import scipy.stats as stats

from scipy.stats import uniform

for index, r in enumerate(results):
    obs = np.array(np.array(r['gesamt']),dtype=float)
    forecasts = np.array(np.array(r.loc[:,'q0.025':'q0.975']),dtype=float)

    # Example quantile forecasts and observations
    quantile_forecasts = forecasts
    observations = obs
    
    # Calculate PIT values
    pit_values = np.zeros_like(quantile_forecasts)
    
    for i in range(quantile_forecasts.shape[1]):
        sorted_forecasts = np.sort(quantile_forecasts[:, i])
        ecdf_values = np.linspace(0, 1, len(sorted_forecasts))
        pit_values[:, i] = np.interp(
            quantile_forecasts[:, i], sorted_forecasts, ecdf_values)
    
    # Create PIT histogram
    fig, ax = plt.subplots()
    ax.hist(pit_values.flatten(), bins=20, density=True,
            alpha=0.75, color='blue', edgecolor='black')
    
    # Plot the uniform distribution for reference
    x = np.linspace(0, 1, 100)
    ax.plot(x, uniform.pdf(x), 'r-', lw=2, label='Uniform [0,1]')
    
    ax.set_title('Probability Integral Transform (PIT)')
    ax.set_xlabel('PIT Values')
    ax.set_ylabel('Density')
    ax.legend()
    
    
    plt.savefig(f'plots/{index} {"".join(random.SystemRandom().choice(string.ascii_uppercase + string.digits) for _ in range(4))}.png')
    plt.show()

### Check Manually

In [ ]:
array = []

for q in ['q0.025', 'q0.25', 'q0.5', 'q0.75', 'q0.975']:
    # print(f'*** {q} ***')
    # arr = models[5]['evaluation'][q] > models[5]['evaluation']['gesamt']
    # print(arr.mean())
    
    per_quantile = []
    
    
    for index, r in enumerate(results):
        # r.loc[:, f'{q}larger'] = r[q] > r['gesamt']
        mean = (r[q] > r['gesamt']).mean()
        # print(r[f'{q}larger'].mean())
        per_quantile.append(round(mean,3))
    per_quantile.append(round((models[model_index]['evaluation'][q] > models[model_index]['evaluation']['gesamt']).mean(), 3))
    
    array.append(per_quantile)
        
pd.DataFrame(array, columns=['36 hour', '40 hour', '44 hour', '60 hour', '64 hour', '68 hour','All horizons'], index=['q0.025', 'q0.25', 'q0.5', 'q0.75', 'q0.975'])



# Lets only use horizon data for training and check again

In [ ]:
df_cval_horizon = df.loc[df.index < start_date_excl]
df_cval_horizon = df_cval_horizon[df_cval_horizon.index.dayofweek.isin([4,5])] # Only keep Friday & Saturday
df_cval_horizon = df_cval_horizon[df_cval_horizon.index.hour.isin([12,16,20])]
df_cval_horizon = df.loc[df.index < start_date_excl]


In [ ]:
df_cval_horizon

In [ ]:

models = [
    # {
    #     'name': 'baseline',
    #     'function': baseline
    # },
    {
        'name': 'model5',
        'function': model5
     }
]
evaluate_models(models, df_cval_horizon, last_x=150, weeks=True)

In [ ]:
df_cval_horizon

In [ ]:
models[0]

In [8]:
from statistics import mean
mean([100,75,50,50,50,75,100])

71.42857142857143

In [10]:
mean([0,100,0,100,50,25,0])

39.285714285714285

In [12]:
(2*120000*600/9)**0.5

4000.0

In [13]:
120/4

30.0

In [15]:
 50*(-1) + 20/1.1 + 40/1.1**2

1.2396694214875978

In [16]:
 75*(-1) + 80/1.1 + 10/1.1**2

5.991735537190074

In [18]:
 round(85*(-1) + 20/1.1 + 70/1.1**2,4)

-8.9669